In [ ]:
import numpy as np
import pandas as pd
import spacy
import nltk
import string
import torch
import torch.nn as nn
from torch import Tensor
import math

### Read the GloVe embeddings

In [ ]:
%run Kaggle_CommonLit_Challenge_TE_Func.ipynb

In [ ]:
# Read the Glove embeddings for each word token
embeddings_dict = {}
# embeddings_dim = 300
# embeddings_dim = 100
embeddings_dim = 200

# with open('C:/Work/Vivek_Work/NLP_Vocab/glove_6B/glove.6B.300d.txt', 'r', encoding="utf-8") as f:
# with open('C:/Work/Vivek_Work/NLP_Vocab/glove_6B/glove.6B.100d.txt', 'r', encoding="utf-8") as f:
with open('C:/Work/Vivek_Work/NLP_Vocab/glove_6B/glove.6B.200d.txt', 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [ ]:
# Number of word tokens
len(embeddings_dict)

In [ ]:
keyset = list(embeddings_dict)
# keyset

### Identify stopwords

In [ ]:
punctuation_labels = list(string.punctuation)
# punctuation_labels

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [ ]:
spacy_stopwords = spacy_stopwords | set(punctuation_labels)
len(spacy_stopwords)

### Generate the word embedding matrix per excerpt

In [ ]:
trng_set = pd.read_csv('C:/Work/Vivek_Work/Challenge_Data_Sets/Kaggle/CommonLit_Readability/train.csv')
trng_set.head()

In [ ]:
trng_set.shape

In [ ]:
trng_set['excerpt'][0]

### Step 1: Identify the common number of words across all excerpts

In [ ]:
non_missing_words_per_excerpt = []
num_excerpts = trng_set.shape[0]

for idx in range(num_excerpts):
    if idx%100 == 0:
        print(idx)
    non_missing_words_per_excerpt.append(excerpt_embedding(trng_set['excerpt'][idx], 0, embeddings_dim))

In [ ]:
pd.Series(non_missing_words_per_excerpt).describe()

In [ ]:
np.unique(np.array(non_missing_words_per_excerpt) >= 77, return_counts=True)

### Step 2: Consider only those excerpts for which the word length exceeds 77

In [ ]:
idx_valid_excerpt = []
num_excerpts = trng_set.shape[0]
# trng_embeddings = np.zeros((1,50))
trng_embeddings = np.zeros((1,embeddings_dim))

for idx in range(num_excerpts):
    if idx%100 == 0:
        print(idx)    
    
    temp = excerpt_embedding(trng_set['excerpt'][idx], 1, embeddings_dim)
    
    if temp.shape[0] >= 77:
        idx_valid_excerpt.append(idx)
        trng_embeddings = np.concatenate((trng_embeddings, temp[0:77, :]), axis=0)

In [ ]:
trng_embeddings.shape

In [ ]:
# Number of input samples for which embeddings could be generated
# trng_embeddings.shape[0] - 1
165242/77

In [ ]:
# Save the excerpt embeddings along with the output values
output_matrix = trng_set.iloc[idx_valid_excerpt, [4, 5]]
output_matrix.to_csv('Scores_Valid_Excerpt.csv', index=False)

In [ ]:
# np.save('Embeddings_Valid_Excerpt_Dim_300.npy', trng_embeddings[1:, :])
# np.save('Embeddings_Valid_Excerpt_Dim_100.npy', trng_embeddings[1:, :])
np.save('Embeddings_Valid_Excerpt_Dim_200.npy', trng_embeddings[1:, :])